<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/answer/3_ロジスティック回帰/3-1_ロジスティック回帰によるガンの悪性腫瘍の分類.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 3-1_ロジスティック回帰によるガンの悪性腫瘍の分類

## ライブラリのインポート

In [1]:
# データを処理するための基本的なライブラリ
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# 訓練データと検証データを分けるライブラリ
from sklearn.model_selection import train_test_split

In [2]:
# ロジスティック回帰モデルをインポート 
from sklearn.linear_model import LogisticRegression

Google Colaboratory上での出力のデフォルト設定

In [3]:
# pandasのDataframeの出力
pd.set_option('display.max_columns', 500) # 表示列の最大
pd.set_option('display.max_rows', 500) # 表示行の最大
pd.set_option('display.unicode.east_asian_width', True) # 日本語出力時にヘッダのずれを解消
pd.options.display.float_format = '{:,.5f}'.format # 表示桁数の設定

# ノートブックの表示桁数設定。この設定はprint文には作用せず、セルの最後に書いたものを出力する際に適用されます。
%precision 3
# numpy配列の指数表示禁止設定
np.set_printoptions(suppress=True)
# numpy配列の表示桁数設定
np.set_printoptions(precision=3)

## データの準備
今回使用するデータはscikit-learnからもデータセットとして利用することができる、UCI ML Breast Cancer Wisconsin (Diagnostic) datasetsのコピーです。<br>
UCI Machine Learning Repositoryから公開されています。<br>
downloaded from : https://goo.gl/U2Uwz2

データセットについての説明はこちらに記載されています。<br>
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

#### データを取り込む
- pandasのread_csvメソッドを使用して、mlho/data/cancer.csvファイルを読み込みます
- 読み込んだものは変数df_cancerに代入します

In [4]:
# csvファイルを読み込みます
df_cancer = pd.read_csv("/content/drive/MyDrive/mlho/data/cancer.csv")

#### データを確認する

In [5]:
# 読み込んだデータを確認します
# Classが目的変数になる腫瘍の悪性または良性を表しています。（0:悪性、1:良性）
df_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Class
0,17.99000,10.38000,122.80000,"1,001.00000",0.11840,0.27760,0.30010,0.14710,0.24190,0.07871,1.09500,0.90530,8.58900,153.40000,0.00640,0.04904,0.05373,0.01587,0.03003,0.00619,25.38000,17.33000,184.60000,"2,019.00000",0.16220,0.66560,0.71190,0.26540,0.46010,0.11890,0
1,20.57000,17.77000,132.90000,"1,326.00000",0.08474,0.07864,0.08690,0.07017,0.18120,0.05667,0.54350,0.73390,3.39800,74.08000,0.00522,0.01308,0.01860,0.01340,0.01389,0.00353,24.99000,23.41000,158.80000,"1,956.00000",0.12380,0.18660,0.24160,0.18600,0.27500,0.08902,0
2,19.69000,21.25000,130.00000,"1,203.00000",0.10960,0.15990,0.19740,0.12790,0.20690,0.05999,0.74560,0.78690,4.58500,94.03000,0.00615,0.04006,0.03832,0.02058,0.02250,0.00457,23.57000,25.53000,152.50000,"1,709.00000",0.14440,0.42450,0.45040,0.24300,0.36130,0.08758,0
3,11.42000,20.38000,77.58000,386.10000,0.14250,0.28390,0.24140,0.10520,0.25970,0.09744,0.49560,1.15600,3.44500,27.23000,0.00911,0.07458,0.05661,0.01867,0.05963,0.00921,14.91000,26.50000,98.87000,567.70000,0.20980,0.86630,0.68690,0.25750,0.66380,0.17300,0
4,20.29000,14.34000,135.10000,"1,297.00000",0.10030,0.13280,0.19800,0.10430,0.18090,0.05883,0.75720,0.78130,5.43800,94.44000,0.01149,0.02461,0.05688,0.01885,0.01756,0.00511,22.54000,16.67000,152.20000,"1,575.00000",0.13740,0.20500,0.40000,0.16250,0.23640,0.07678,0


In [6]:
# df_cancerのデータ要約を確認
df_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [7]:
# df_cancerの統計情報を確認
df_cancer.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Class
count,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000,569.00000
mean,14.12729,19.28965,91.96903,654.88910,0.09636,0.10434,0.08880,0.04892,0.18116,0.06280,0.40517,1.21685,2.86606,40.33708,0.00704,0.02548,0.03189,0.01180,0.02054,0.00379,16.26919,25.67722,107.26121,880.58313,0.13237,0.25427,0.27219,0.11461,0.29008,0.08395,0.62742
std,3.52405,4.30104,24.29898,351.91413,0.01406,0.05281,0.07972,0.03880,0.02741,0.00706,0.27731,0.55165,2.02185,45.49101,0.00300,0.01791,0.03019,0.00617,0.00827,0.00265,4.83324,6.14626,33.60254,569.35699,0.02283,0.15734,0.20862,0.06573,0.06187,0.01806,0.48392
min,6.98100,9.71000,43.79000,143.50000,0.05263,0.01938,0.00000,0.00000,0.10600,0.04996,0.11150,0.36020,0.75700,6.80200,0.00171,0.00225,0.00000,0.00000,0.00788,0.00089,7.93000,12.02000,50.41000,185.20000,0.07117,0.02729,0.00000,0.00000,0.15650,0.05504,0.00000
25%,11.70000,16.17000,75.17000,420.30000,0.08637,0.06492,0.02956,0.02031,0.16190,0.05770,0.23240,0.83390,1.60600,17.85000,0.00517,0.01308,0.01509,0.00764,0.01516,0.00225,13.01000,21.08000,84.11000,515.30000,0.11660,0.14720,0.11450,0.06493,0.25040,0.07146,0.00000
50%,13.37000,18.84000,86.24000,551.10000,0.09587,0.09263,0.06154,0.03350,0.17920,0.06154,0.32420,1.10800,2.28700,24.53000,0.00638,0.02045,0.02589,0.01093,0.01873,0.00319,14.97000,25.41000,97.66000,686.50000,0.13130,0.21190,0.22670,0.09993,0.28220,0.08004,1.00000
75%,15.78000,21.80000,104.10000,782.70000,0.10530,0.13040,0.13070,0.07400,0.19570,0.06612,0.47890,1.47400,3.35700,45.19000,0.00815,0.03245,0.04205,0.01471,0.02348,0.00456,18.79000,29.72000,125.40000,"1,084.00000",0.14600,0.33910,0.38290,0.16140,0.31790,0.09208,1.00000
max,28.11000,39.28000,188.50000,"2,501.00000",0.16340,0.34540,0.42680,0.20120,0.30400,0.09744,2.87300,4.88500,21.98000,542.20000,0.03113,0.13540,0.39600,0.05279,0.07895,0.02984,36.04000,49.54000,251.20000,"4,254.00000",0.22260,1.05800,1.25200,0.29100,0.66380,0.20750,1.00000


#### 説明変数、目的変数を切り出す

In [8]:
# 目的変数にするClass以外をすべて説明変数にする
x = df_cancer.drop(columns='Class')

In [9]:
x.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99000,10.38000,122.80000,"1,001.00000",0.11840,0.27760,0.30010,0.14710,0.24190,0.07871,1.09500,0.90530,8.58900,153.40000,0.00640,0.04904,0.05373,0.01587,0.03003,0.00619,25.38000,17.33000,184.60000,"2,019.00000",0.16220,0.66560,0.71190,0.26540,0.46010,0.11890
1,20.57000,17.77000,132.90000,"1,326.00000",0.08474,0.07864,0.08690,0.07017,0.18120,0.05667,0.54350,0.73390,3.39800,74.08000,0.00522,0.01308,0.01860,0.01340,0.01389,0.00353,24.99000,23.41000,158.80000,"1,956.00000",0.12380,0.18660,0.24160,0.18600,0.27500,0.08902


In [10]:
y = df_cancer['Class']

In [11]:
y.head(2)

0    0
1    0
Name: Class, dtype: int64

#### データを訓練データと検証データに分割する

**本研修でbreast-cancerデータセットを使用する際には、訓練データ70%、random_state=3で固定しています。他のモデルとの比較をしやすくするためです**

In [12]:
# 訓練データと検証データに分割(70%を訓練用に使用)
# stratifyは指定した列の値が均等に割り振られる
train_x, val_x, train_y, val_y = train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=3, stratify=y)

## モデルの定義

In [13]:
model = LogisticRegression(max_iter=2500)

## モデルの学習

In [14]:
model.fit(train_x, train_y)

LogisticRegression(max_iter=2500)

## 評価
評価はロジスティック回帰のモデルに用意されているscoreメソッドで行います。

精度の計算方法は分類なのか回帰なのかによって異なりますが、線形回帰の時と同じくscoreメソッドを用います。<br>
scikit-learnに実装されている各モデルのscoreメソッドは、分類の場合は正解率、回帰の場合は決定係数R2で計算されるようになっています。

In [15]:
# 訓練データで予測精度（正解率）を確認する
model.score(train_x, train_y)

0.960

In [16]:
# 検証データで予測精度（正解率）を確認する
model.score(val_x, val_y)

0.953

In [17]:
# 検証データで予測し、最初の5件の分類確立を確認してみる
model.predict_proba(val_x)[:5]

array([[0.   , 1.   ],
       [0.006, 0.994],
       [0.006, 0.994],
       [0.001, 0.999],
       [0.542, 0.458]])

In [18]:
# 最初の5件の検証データで予測結果を表示(malignant:悪性(0), benign:良性(1))
model.predict(val_x)[:5]

array([1, 1, 1, 1, 0])

In [19]:
# 正解を確認する
val_y.head()

153    1
48     1
163    1
371    1
340    1
Name: Class, dtype: int64

## 予測精度の改善

### データのスケールを揃える
標準化
- データを平均が0、標準偏差(データのばらつき具合)が1になるように調整する
- スケールを揃える為の方法として良く利用される

In [20]:
# 訓練データ説明変数の各列の平均を計算する
train_x_mean = train_x.mean()
train_x_mean.head()

mean radius        14.08899
mean texture       19.24535
mean perimeter     91.70073
mean area         651.78040
mean smoothness     0.09609
dtype: float64

In [21]:
# 訓練データ説明変数の各列の標準偏差を計算する
train_x_std = train_x.std()
train_x_std.head()

mean radius         3.52637
mean texture        4.25211
mean perimeter     24.35110
mean area         352.01484
mean smoothness     0.01430
dtype: float64

In [22]:
# 訓練データの標準化を行う
train_x_scaled = (train_x - train_x_mean) / train_x_std
train_x_scaled.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
387,-0.05926,-0.72561,-0.13678,-0.15676,-1.80654,-1.04197,-0.85225,-1.03515,-0.72057,-1.10538,-0.56120,-1.05516,-0.58867,-0.38424,-1.07068,-0.63320,-0.43679,-0.80415,-0.48117,-0.70788,-0.14193,-0.92195,-0.21060,-0.22270,-2.04497,-0.81781,-0.78022,-1.05492,-0.55655,-0.96313
225,0.07118,-1.35823,0.03323,-0.03006,0.20798,-0.51101,-0.38549,-0.05711,0.95663,-1.14188,0.44814,-0.71937,0.47043,0.19456,-0.00128,-0.62594,-0.29984,0.15984,0.23247,-0.90485,0.12091,-1.42588,0.10999,0.00348,-0.09552,-0.63108,-0.51586,-0.07917,0.26578,-1.27192
430,0.22998,0.77248,0.42706,0.09437,0.23666,2.26950,2.35570,1.27430,0.83478,0.89351,-0.56535,-0.60856,0.31729,-0.35963,-0.04074,2.04925,1.74927,1.72361,-0.65180,0.77385,0.03330,0.32555,0.55774,-0.06814,0.43465,2.92750,3.09384,2.05937,-0.04418,1.79803
324,-0.53567,-0.94902,-0.56222,-0.55077,-0.65449,-0.71613,-0.85872,-0.81586,-0.60948,-0.18595,-0.54839,-0.72784,-0.45976,-0.47875,-0.53773,-0.63153,-0.78174,-1.11327,-0.84318,-0.67098,-0.50908,-0.69051,-0.46582,-0.50954,-0.27369,-0.37338,-0.74308,-0.89792,-0.36836,-0.21330
314,-1.55740,-0.15177,-1.54452,-1.22319,0.79135,-0.84883,-1.11170,-1.25688,1.27200,1.54062,-0.24962,2.74770,-0.32415,-0.50634,4.34533,-0.63991,-1.16867,-1.96147,4.70570,1.17634,-1.50996,-0.51651,-1.49444,-1.11611,0.12177,-1.10959,-1.31334,-1.75395,0.39229,-0.12643


In [23]:
# 検証データの標準化を行う
val_x_scaled = (val_x - train_x_mean) / train_x_std
val_x_scaled.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
153,-0.83343,-1.44995,-0.85543,-0.76667,0.10166,-0.98836,-0.86024,-0.79136,0.07861,-0.21964,-0.67046,-0.77337,-0.73305,-0.55992,0.61280,-0.92197,-0.72836,-0.60356,0.37312,-0.84425,-0.87622,-1.52436,-0.90939,-0.76119,0.09569,-1.03260,-0.96562,-0.90562,-0.05525,-0.87963
48,-0.57821,-1.08543,-0.56099,-0.57520,0.49057,-0.23193,-0.27536,-0.54035,-0.47688,-0.30667,-0.52541,-0.86530,-0.51700,-0.45897,-0.51069,-0.62650,-0.31383,-1.02783,-0.73366,-0.53228,-0.50699,-0.80213,-0.50253,-0.51043,0.76057,-0.22759,0.17706,-0.74508,-0.23236,-0.02276
163,-0.49597,0.69957,-0.48666,-0.53202,0.35767,-0.03080,-0.43040,-0.52132,-0.92126,0.70120,-0.40748,0.76969,-0.46183,-0.42034,1.35128,0.35077,-0.01820,-0.08924,-0.21370,0.60447,-0.54454,0.50775,-0.57775,-0.56277,0.57806,-0.11120,-0.48849,-0.49147,-0.98985,0.41493
371,0.31222,-1.41938,0.24431,0.17050,-1.15112,-0.64218,-0.68123,-0.56433,-0.31202,-1.00712,-0.84679,-1.42447,-0.77997,-0.50841,-0.66214,-0.59407,-0.59767,-0.45131,-0.48117,-0.78675,0.00201,-1.61793,-0.06613,-0.09219,-0.83862,-0.49552,-0.64779,-0.49394,-0.64352,-0.88299
340,0.09387,-0.63624,0.10058,-0.03006,0.09956,0.20494,-0.09584,-0.15615,0.37248,0.21130,-0.20328,-0.79260,-0.08954,-0.17682,-0.80054,0.28256,0.24502,0.08891,-0.22062,-0.16282,0.10005,-0.66917,0.13984,-0.01615,-0.10855,0.54936,0.52156,0.42465,0.25154,0.23672


In [24]:
# ロジスティック回帰分析の実行
model.fit(train_x_scaled, train_y)

LogisticRegression(max_iter=2500)

In [25]:
# 訓練データで予測精度（正解率）を確認する
model.score(train_x_scaled, train_y)

0.987

In [26]:
# 検証データで予測精度（正解率）を確認する
model.score(val_x_scaled, val_y)

0.988

## 精度以外の評価指標を確認する
- 適合率（precision）：正と予測したものが、どれだけ正しかったか（高いほど無駄うちが少ない）
- 再現率（recall）： 実際に正であったもののうち、どれだけ正と予測できたか（高いほど取りこぼしが少ない）
- f値（f1-score）：適合率と再現率の調和平均（F値が高い＝適合率も再現率も高い）

In [27]:
# まとめて評価指標を確認できるclassification_reportをインポート
from sklearn.metrics import classification_report

In [28]:
# まずは予測を行う
prediction = model.predict(val_x_scaled)

In [29]:
# classification_reportでまとめて評価指標を確認
# printを使用することでフォーマットが整形されて出力される
print(classification_report(val_y, prediction))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        64
           1       0.99      0.99      0.99       107

    accuracy                           0.99       171
   macro avg       0.99      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171



- macro avgはマクロ平均。陽性・陰性を入れ替えて算出した値の平均。
- weighted avg は陽性・陰性を変えて算出した値の加重平均。正解クラスの個数に応じて加重平均する。

（補足）どこから正例とみなすかの閾値はプログラミングで調整をすることができます。predictメソッドは分類確率が0.5より高い方を結果として返しますが、predict_probaメソッドは分類確率自体を返します。例えば、predict_pobaメソッドの結果が0.6以下を0と出力するようにif文でプログラミングすれば、0と判断（悪性の腫瘍と判断される）されやすくして取りこぼしを少なくすることができます。

このノートブックは以上です。